In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 00:37:53 WARN Utils: Your hostname, queen resolves to a loopback address: 127.0.1.1; using 192.168.1.124 instead (on interface wlo1)
24/03/03 00:37:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 00:37:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY 
    1, 2
```

In [6]:
df_green.rdd.take(2)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 29, 14, 37, 24), store_and_fwd_flag='N', RatecodeID=1, PULocationID=97, DOLocationID=97, passenger_count=1, trip_distance=0.81, fare_amount=5.5, extra=0.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=7.3, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), lpep_dropoff_datetime=datetime.datetime(2020, 1, 24, 4, 46, 50), store_and_fwd_flag='N', RatecodeID=5, PULocationID=7, DOLocationID=141, passenger_count=1, trip_distance=3.02, fare_amount=40.0, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.0, total_amount=42.75, payment_type=1, trip_type=2, congestion_surcharge=2.75)]

In [7]:
df_green.take(2)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), lpep_dropoff_datetime=datetime.datetime(2020, 1, 29, 14, 37, 24), store_and_fwd_flag='N', RatecodeID=1, PULocationID=97, DOLocationID=97, passenger_count=1, trip_distance=0.81, fare_amount=5.5, extra=0.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=7.3, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), lpep_dropoff_datetime=datetime.datetime(2020, 1, 24, 4, 46, 50), store_and_fwd_flag='N', RatecodeID=5, PULocationID=7, DOLocationID=141, passenger_count=1, trip_distance=3.02, fare_amount=40.0, extra=0.0, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.0, total_amount=42.75, payment_type=1, trip_type=2, congestion_surcharge=2.75)]

In [8]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [9]:
rdd.take(2)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), PULocationID=97, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), PULocationID=7, total_amount=42.75)]

## Implementing Filter

In [10]:
from datetime import datetime

In [11]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [12]:
rdd.filter(filter_outliers).take(2)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), PULocationID=97, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 4, 36, 44), PULocationID=7, total_amount=42.75)]

## Group By

### Prepare for grouping

In [13]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 14, 32), PULocationID=97, total_amount=7.3)

In [14]:
row.lpep_pickup_datetime

datetime.datetime(2020, 1, 29, 14, 32)

In [15]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 29, 14, 0)

In [16]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)

    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [17]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(2)

[((datetime.datetime(2020, 1, 29, 14, 0), 97), (7.3, 1)),
 ((datetime.datetime(2020, 1, 24, 4, 0), 7), (42.75, 1))]

### Reduce by key

In [18]:
def calculate_revenue(left_value, right_value):
    l_amount, l_count = left_value
    r_amount, r_count = right_value
    
    return (l_amount+r_amount, l_count+r_count)

In [19]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(2)

[((datetime.datetime(2020, 1, 26, 11, 0), 166), (373.69000000000005, 23)),
 ((datetime.datetime(2020, 1, 30, 17, 0), 82), (503.90000000000015, 37))]

### Unnest/unwrap the tuples

In [20]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(lambda row: (row[0][0], row[0][1], row[1][0], row[1][1])) \
    .take(2)

[(datetime.datetime(2020, 1, 26, 11, 0), 166, 373.69000000000005, 23),
 (datetime.datetime(2020, 1, 30, 17, 0), 82, 503.90000000000015, 37)]

In [21]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(lambda row: (row[0][0], row[0][1], row[1][0], row[1][1])) \
    .toDF() \
    .take(2)

[Row(_1=datetime.datetime(2020, 1, 26, 11, 0), _2=166, _3=373.69000000000005, _4=23),
 Row(_1=datetime.datetime(2020, 1, 30, 17, 0), _2=82, _3=503.90000000000015, _4=37)]

In [22]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(lambda row: (row[0][0], row[0][1], row[1][0], row[1][1])) \
    .toDF() \
    .show(2)

+-------------------+---+------------------+---+
|                 _1| _2|                _3| _4|
+-------------------+---+------------------+---+
|2020-01-26 11:00:00|166|373.69000000000005| 23|
|2020-01-30 17:00:00| 82|503.90000000000015| 37|
+-------------------+---+------------------+---+
only showing top 2 rows



In [23]:
from collections import namedtuple

In [24]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [25]:
df_result= rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF()

In [26]:
df_result.show(2)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-26 11:00:00| 166|373.69000000000005|   23|
|2020-01-30 17:00:00|  82|503.90000000000015|   37|
+-------------------+----+------------------+-----+
only showing top 2 rows



### Set schema

In [27]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', LongType(), True), StructField('revenue', DoubleType(), True), StructField('count', LongType(), True)])

In [28]:
from pyspark.sql import types

In [29]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [30]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [32]:
df_result.write.parquet('tmp/green-revenue', mode='overwrite')

24/03/03 00:42:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/03 00:42:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/03 00:42:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/03 00:42:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 00:42:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/03 00:42:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/03 00:42:29 WARN MemoryManager: Total allocation exceeds 95.

## Map Partition

In [45]:
import pandas as pd

In [46]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [53]:
df = pd.DataFrame(rows)

In [54]:
df.itertuples()

In [55]:
list(df.itertuples())

[Pandas(Index=0, _1=Timestamp('2020-01-29 14:32:00'), _2=97, _3=7.3),
 Pandas(Index=1, _1=Timestamp('2020-01-24 04:36:44'), _2=7, _3=42.75),
 Pandas(Index=2, _1=Timestamp('2020-01-23 13:52:00'), _2=43, _3=8.3),
 Pandas(Index=3, _1=Timestamp('2020-01-26 22:18:03'), _2=260, _3=6.8),
 Pandas(Index=4, _1=Timestamp('2020-01-04 16:00:00'), _2=177, _3=19.78),
 Pandas(Index=5, _1=Timestamp('2020-01-21 23:02:04'), _2=24, _3=10.56),
 Pandas(Index=6, _1=Timestamp('2020-01-25 19:09:59'), _2=82, _3=15.3),
 Pandas(Index=7, _1=Timestamp('2020-01-24 21:30:37'), _2=260, _3=7.8),
 Pandas(Index=8, _1=Timestamp('2020-01-03 11:37:21'), _2=130, _3=4.3),
 Pandas(Index=9, _1=Timestamp('2020-01-09 07:06:00'), _2=218, _3=12.4)]

In [57]:
df = pd.DataFrame(rows, columns=['lpep_pickup_datetime', 'PULocationID', 'total_amount'])

list(df.itertuples())

[Pandas(Index=0, lpep_pickup_datetime=Timestamp('2020-01-29 14:32:00'), PULocationID=97, total_amount=7.3),
 Pandas(Index=1, lpep_pickup_datetime=Timestamp('2020-01-24 04:36:44'), PULocationID=7, total_amount=42.75),
 Pandas(Index=2, lpep_pickup_datetime=Timestamp('2020-01-23 13:52:00'), PULocationID=43, total_amount=8.3),
 Pandas(Index=3, lpep_pickup_datetime=Timestamp('2020-01-26 22:18:03'), PULocationID=260, total_amount=6.8),
 Pandas(Index=4, lpep_pickup_datetime=Timestamp('2020-01-04 16:00:00'), PULocationID=177, total_amount=19.78),
 Pandas(Index=5, lpep_pickup_datetime=Timestamp('2020-01-21 23:02:04'), PULocationID=24, total_amount=10.56),
 Pandas(Index=6, lpep_pickup_datetime=Timestamp('2020-01-25 19:09:59'), PULocationID=82, total_amount=15.3),
 Pandas(Index=7, lpep_pickup_datetime=Timestamp('2020-01-24 21:30:37'), PULocationID=260, total_amount=7.8),
 Pandas(Index=8, lpep_pickup_datetime=Timestamp('2020-01-03 11:37:21'), PULocationID=130, total_amount=4.3),
 Pandas(Index=9, l

In [49]:
def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [50]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [51]:
duration_rdd.mapPartitions(apply_model_in_batch).collect()

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 14:32:00'), PULocationID=97, DOLocationID=97, trip_distance=0.81, predicted_duration=4.050000000000001),
 Pandas(Index=1, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-24 04:36:44'), PULocationID=7, DOLocationID=141, trip_distance=3.02, predicted_duration=15.1),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:52:00'), PULocationID=43, DOLocationID=24, trip_distance=1.32, predicted_duration=6.6000000000000005),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-26 22:18:03'), PULocationID=260, DOLocationID=260, trip_distance=0.89, predicted_duration=4.45),
 Pandas(Index=4, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-04 16:00:00'), PULocationID=177, DOLocationID=49, trip_distance=2.83, predicted_duration=14.15),
 Pandas(Index=5, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-21 23:02:04'), PULocationID=24, DOLocationID=41, trip_distance=1.33,

In [58]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [59]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
| 4.050000000000001|
|              15.1|
|6.6000000000000005|
|              4.45|
|             14.15|
|              6.65|
|             17.05|
|               7.5|
|               1.0|
|12.350000000000001|
|              97.6|
|               2.6|
|              40.9|
|             14.75|
|               7.9|
|               4.3|
|              79.2|
| 6.550000000000001|
|3.5999999999999996|
|               5.5|
+------------------+
only showing top 20 rows

